In [ ]:
import gdown
import pandas as pd
import numpy as np
import os
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import logging
from datetime import date, timedelta
import datetime
import re

In [ ]:
url = "https://drive.google.com/drive/folders/1tq2wthWD_eFpuT_cJwLs4rxDs0h1OfBd"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/data_quality_rk_clss/book_master_2023-03-24.csv']

In [ ]:
#https://medium.com/analytics-vidhya/how-to-connect-google-drive-to-python-using-pydrive-9681b2a14f20
# https://pythonhosted.org/PyDrive/oauth.html#authentication-in-two-lines
# gauth = GoogleAuth()
# Create local webserver and auto handles authentication.
# gauth.LocalWebserverAuth()

# **** Data Transformation Part ****

In [ ]:
# https://www.w3schools.com/python/python_datetime.asp
today=date.today().strftime("%Y-%m-%d") #2022-10-29
yesterday=date.today()- datetime.timedelta(days=160)
yesterday=yesterday.strftime("%Y-%m-%d")
designatedFileName='book_master_'+yesterday+'.csv' # to fix this
# designatedFileName='book_master_2023-03-24.csv' # hardcode
print(designatedFileName)

book_master_2023-03-24.csv


In [ ]:
today

'2023-08-31'

In [ ]:
# https://docs.python.org/3/library/logging.html
log_file_name=today+'_log.log'

logging.basicConfig(filename=log_file_name,
                    format='%(asctime)s %(message)s',
                    filemode='w',level=logging.INFO)

logger = logging.getLogger()

In [ ]:
df=pd.read_csv(f'data_quality_rk_clss/{designatedFileName}')
df.tail(3)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,2022-09-25,NaN
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,2022-09-25,NaN
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN


In [ ]:
# file_list = drive.ListFile({'q': "'1tq2wthWD_eFpuT_cJwLs4rxDs0h1OfBd' in parents and trashed=false"}).GetList()
# file_df=pd.DataFrame(file_list)

#Check if the file is existed

# file_df[file_df.title==designatedFileName]['id'].values[0]

# if(file_df[file_df.title==designatedFileName].shape[0]!=0):
#     print(f'{designatedFileName}` detected')
#     logger.info(f'`{designatedFileName}` detected')
# else:
#     logger.error(f'`{designatedFileName}` was not found')
#     raise ImportError(f'`{designatedFileName}` was not found')

In [ ]:
# # https://pythonhosted.org/PyDrive/filemanagement.html#download-file-content

# designatedID=file_df[file_df.title==designatedFileName]['id'].values[0]
# tmpFileName='tmp_'+designatedFileName
# drive.CreateFile({'id': designatedID}).GetContentFile(tmpFileName)
# df=pd.read_csv(tmpFileName)

## 1. REMOVE QUANTITY <=0

In [ ]:
#create df that has no quantity<=0
df=df[df.Quantity>0]

In [ ]:
df[df.Quantity==0]

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date


In [ ]:
df[['Title']][df.Quantity==0]

,Title


## 2. QC book title can't have multiple rows

In [ ]:
#group by book title and count row
df.groupby('Title').count()
g=df.groupby('Title').count()
g

,book_id,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
Title,,,,,,,,,,
"""Most Blessed of the Patriarchs"": Thomas Jefferson and the Empire of the Imagination",1,1,1,1,1,1,1,1,1,0
#GIRLBOSS,1,1,1,1,1,1,1,1,1,0
#HigherSelfie: Wake Up Your Life. Free Your Soul. Find Your Tribe.,1,1,1,1,1,1,1,1,1,0
'Salem's Lot,1,1,1,1,1,1,1,1,1,0
(Un)Qualified: How God Uses Broken People to Do Big Things,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...
Zealot: The Life and Times of Jesus of Nazareth,1,1,1,1,1,1,1,1,1,0
Zero History (Blue Ant #3),1,1,1,1,1,1,1,1,1,0
"Zero to One: Notes on Startups, or How to Build the Future",1,1,1,1,1,1,1,1,1,0


In [ ]:
# g[g.Book_category>1]
g[g['Book_category']>1]

,book_id,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
Title,,,,,,,,,,
America's War for the Greater Middle East: A Military History,2,2,2,2,2,2,2,2,2,0
The Star-Touched Queen,2,2,2,2,2,2,2,2,2,0


In [ ]:
g[g['Book_category']>1].index.tolist()

["America's War for the Greater Middle East: A Military History",
 'The Star-Touched Queen']

In [ ]:
#add dup title to dup_title
dup_title = g[g['Book_category']>1].index.tolist()
dup_title

["America's War for the Greater Middle East: A Military History",
 'The Star-Touched Queen']

In [ ]:
df[df.Title.isin(dup_title)]

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
0,3000,A Light in the Attic,Poetry,Three,51.77,In stock,22,M.Bridges,Dragonsteel,2022-09-25,NaN
1,3001,Tipping the Velvet,Historical Fiction,One,53.74,In stock,20,E. Barron,Dragonsteel,2022-09-25,NaN
2,3002,Soumission,Fiction,One,50.10,In stock,20,E. Barron,TOR,2022-09-25,NaN
3,3003,Sharp Objects,Mystery,Four,47.82,In stock,20,E. Barron,Dragonsteel,2022-09-25,NaN
4,3004,Sapiens: A Brief History of Humankind,History,Five,54.23,In stock,20,E. Barron,Dragonsteel,2022-09-25,NaN
...,...,...,...,...,...,...,...,...,...,...,...
996,3996,Alice in Wonderland (Alice's Adventures in Won...,Classics,One,55.53,In stock,1,E. Barron,Dragonsteel,2022-09-25,NaN
997,3997,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Sequential Art,Four,57.06,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,2022-09-25,NaN
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,2022-09-25,NaN


In [ ]:
#remove title which has multiple rows (by randomly)
df=df.drop_duplicates(subset=['Title'])

In [ ]:
df[df.Title.isin(dup_title)]


,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
236,3236,The Star-Touched Queen,Fantasy,Five,46.02,In stock,14,E. Barron,O' Reilly,2022-09-25,NaN
639,3639,America's War for the Greater Middle East: A M...,History,Two,51.22,In stock,4,M.Bridges,Dragonsteel,2022-09-24,NaN


## 3. Rename Thai column

In [ ]:
df.tail(3)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,2022-09-25,NaN
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,2022-09-25,NaN
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN


In [ ]:
#rename สำนักพิมพ์ to Publisher

df.rename(columns = {'สำนักพิมพ์':'Publisher'}, inplace = True)
# df=df.rename(columns = {'สำนักพิมพ์':'Publisher'})
df.tail(3)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,2022-09-25,NaN
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,2022-09-25,NaN
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN


## 4. Amend Thai value inside the cell to English

In [ ]:
df.Book_category.value_counts()

Default               152
Nonfiction            108
Sequential Art         75
Add a comment          67
Fiction                64
Young Adult            48
Fantasy                47
Romance                35
Mystery                32
Food and Drink         30
Childrens              29
Historical Fiction     26
Classics               19
Poetry                 18
History                18
Horror                 17
Womens Fiction         17
Science Fiction        16
Science                14
Music                  13
Business               12
Thriller               11
Travel                 11
Humor                  10
Philosophy             10
Autobiography           9
Art                     8
Psychology              7
Religion                7
Spirituality            6
Christian Fiction       6
New Adult               6
Sports and Games        5
YA                      5
Biography               5
Self Help               5
Health                  4
Contemporary            3
Politics    

In [ ]:
df[df.Book_category=='กวี']
#df.loc[df.Book_category=='กวี']

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date
40,3040,Slow States of Collapse: Poems,กวี,Three,57.31,In stock,17,A. Edwards,Dragonsteel,2022-09-25,NaN


In [ ]:
#change row that has [กวี] value to [Poetry] value
df['Book_category']=np.where(df.Book_category=='กวี', 'Poetry', df.Book_category)


In [ ]:
#check result
df.loc[df.book_id==3040]

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date
40,3040,Slow States of Collapse: Poems,Poetry,Three,57.31,In stock,17,A. Edwards,Dragonsteel,2022-09-25,NaN


## 5. Create column: [import_date] = [today] value


In [ ]:
# set import_date = today
df['import_date']=today
df.tail()

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date,import_date
996,3996,Alice in Wonderland (Alice's Adventures in Won...,Classics,One,55.53,In stock,1,E. Barron,Dragonsteel,2022-09-25,NaN,2023-03-25
997,3997,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Sequential Art,Four,57.06,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN,2023-03-25
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,2022-09-25,NaN,2023-03-25
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,2022-09-25,NaN,2023-03-25
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN,2023-03-25


In [ ]:
df['import_date'].unique()

array(['2023-03-25'], dtype=object)

## 6. check date column to [YYYY-MM-DD] date format DD/MM/YYYY e.g. 25/09/2022 to 2022-09-25



In [ ]:
#see this doc: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

In [ ]:
df.loc[df.book_id.isin([3665,3040])]

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date,import_date
40,3040,Slow States of Collapse: Poems,Poetry,Three,57.31,In stock,17,A. Edwards,Dragonsteel,2022-09-25,NaN,2023-03-25
665,3665,No One Here Gets Out Alive,Music,Five,20.02,In stock,4,E. Barron,TOR,25/09/2022,NaN,2023-03-25


In [ ]:
#  2/2/2023 M D Y ? D M Y

25/9/2022
9/25/20222

In [ ]:
#reformat [PublishedDate] with new column name: [PublishedDate_cleaned]
df['PublishedDate_cleaned']=pd.to_datetime(df['PublishedDate']).dt.strftime('%Y-%m-%d')

/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '25/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)


In [ ]:
df[['PublishedDate','PublishedDate_cleaned']].value_counts()

PublishedDate  PublishedDate_cleaned
2022-09-25     2022-09-25               984
2022-09-21     2022-09-21                 6
25/09/2022     2022-09-25                 3
2022-09-24     2022-09-24                 1
dtype: int64

In [ ]:
df.drop(columns='PublishedDate').head(2)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,add_date,import_date,PublishedDate_cleaned
0,3000,A Light in the Attic,Poetry,Three,51.77,In stock,22,M.Bridges,Dragonsteel,NaN,2023-03-25,2022-09-25
1,3001,Tipping the Velvet,Historical Fiction,One,53.74,In stock,20,E. Barron,Dragonsteel,NaN,2023-03-25,2022-09-25


In [ ]:
df.head()

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date,import_date,PublishedDate_cleaned
0,3000,A Light in the Attic,Poetry,Three,51.77,In stock,22,M.Bridges,Dragonsteel,2022-09-25,NaN,2023-03-25,2022-09-25
1,3001,Tipping the Velvet,Historical Fiction,One,53.74,In stock,20,E. Barron,Dragonsteel,2022-09-25,NaN,2023-03-25,2022-09-25
2,3002,Soumission,Fiction,One,50.10,In stock,20,E. Barron,TOR,2022-09-25,NaN,2023-03-25,2022-09-25
3,3003,Sharp Objects,Mystery,Four,47.82,In stock,20,E. Barron,Dragonsteel,2022-09-25,NaN,2023-03-25,2022-09-25
4,3004,Sapiens: A Brief History of Humankind,History,Five,54.23,In stock,20,E. Barron,Dragonsteel,2022-09-25,NaN,2023-03-25,2022-09-25


In [ ]:
# Drop column:[PublishedDate]
df.drop(columns='PublishedDate',inplace=True)
# df=df.drop(columns='PublishedDate')
# Rename [PublishedDate_cleaned] to [PublishedDate]
df.rename(columns = {'PublishedDate_cleaned':'PublishedDate'},inplace=True)

In [ ]:
df.tail(3)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,add_date,import_date,PublishedDate
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,NaN,2023-03-25,2022-09-25
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,NaN,2023-03-25,2022-09-25
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,E. Barron,O' Reilly,NaN,2023-03-25,2022-09-25


## 7. Select certain columns

In [ ]:
df.columns

Index(['book_id', 'Title', 'Book_category', 'Star_rating', 'Price', 'Stock',
       'Quantity', 'user_input', 'Publisher', 'add_date', 'import_date',
       'PublishedDate'],
      dtype='object')

In [ ]:
df=df[['book_id',
 'Title',
 'Book_category',
 'Star_rating',
 'Price',
 'Stock',
 'Quantity',
 'Publisher','PublishedDate','import_date']]

In [ ]:
df.tail(3)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,Publisher,PublishedDate,import_date
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,Dragonsteel,2022-09-25,2023-03-25
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,O' Reilly,2022-09-25,2023-03-25
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,O' Reilly,2022-09-25,2023-03-25


 ### ----- Transformation Part Completed -----

# **** Data Quality Part ****

## 1.Check template column. Must be the same

In [ ]:
template_col=['book_id','Title','Book_category','Star_rating','Price','Stock','Quantity','Publisher','PublishedDate','import_date']

# template_col

In [ ]:
df.columns.tolist()

['book_id',
 'Title',
 'Book_category',
 'Star_rating',
 'Price',
 'Stock',
 'Quantity',
 'Publisher',
 'PublishedDate',
 'import_date']

In [ ]:
if ((template_col==df.columns.tolist()))==False:

    logger.error("File columns don't match the current table format")
    logger.error(f"Unsuccessful:File columns don't match the current table format \n template: {template_col} \n csv file: {df.columns} ")
    raise ValueError(f"Unsuccessful:File columns don't match the current table format \n template: {template_col} \n csv file: {df.columns} ")

else:
    print('File columns match the current table format')
    logger.info('File columns match the current table format')

File columns match the current table format


1.1 Check if there is quantity <=0

In [ ]:
len(df[df.Quantity<=0])

0

In [ ]:
if (len(df[df.Quantity<=0])>1):
    raise ImportError(f'`{df}` quantity has <=0')
else:
    print('passs')

passs


##  2. book title can't have multiple rows

In [ ]:
g=df.groupby('Title').count()
g
if(len(g[g.Book_category>1]>1)):
    err_txt='Book_category>1 per title'
    logger.error(err_txt)
    raise ImportError(err_txt)
else:
    print('pass')

pass


## 3. If import_date = today

True

In [ ]:
pd.Series(df.import_date==today).all()==True

True

In [ ]:
if(pd.Series(df.import_date==today).all()==False):

    raise ImportError('update date is not today')
else:
    print('pass')

pass


## -------------- END --------------

## Data Transformation + DQC were done, then what??

In [ ]:
import push_to_db as pppp
ppp.push('sfsdgf')